In [23]:
from __future__ import print_function
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
import torchnet as tnt

from torchvision import datasets, transforms, models
# from Places205 import Places205
import numpy as np
import random
from torch.utils.data.dataloader import default_collate
from PIL import Image
import os
import os.path as osp
import sys
import copy
from tqdm import tqdm

%matplotlib inline 
from matplotlib import pyplot as plt

from adversarial_contrastive_model import resnet18

**NT-Xent Begins**

In [2]:
# data_dir = '/data/majie/Datasets/TinyDataset/hymenoptera_data'
# ant = osp.join(data_dir, 'train', 'ants', 'VietnameseAntMimicSpider.jpg')
# bee = osp.join(data_dir, 'train', 'bees', '39747887_42df2855ee.jpg')

In [3]:
# def load_image(file_path):
#     image = Image.open(file_path).convert('RGB')
# #     image = np.asarray(image)
#     return image # PIL

In [4]:
# transform_crop = transforms.Compose([transforms.RandomCrop(224), transforms.ToTensor()])
# transform_color = transforms.Compose([transforms.Resize((224, 224)), 
#                                       transforms.ColorJitter(1, 0.1, 0.05, 0.0), 
#                                       transforms.ToTensor()])

In [5]:
# ant_crop = transform_crop(load_image(ant))
# ant_color = transform_color(load_image(ant)) # torch.Size([3, 224, 224])

In [6]:
# bee_crop = transform_crop(load_image(bee))
# bee_color = transform_color(load_image(bee)) # torch.Size([3, 224, 224])

In [7]:
# ant1 = ant_crop.view(1, -1)
# ant2 = ant_color.view(1, -1)
# bee1 = bee_crop.view(1, -1)
# bee2 = bee_color.view(1, -1)

In [8]:
animals = [ant1, ant2, bee1, bee2]

NameError: name 'ant1' is not defined

In [ ]:
# 构造 L2 distance 矩阵， 计算 每张 图片 与 所有图片的距离 N * N

In [ ]:
# s1 = [F.pairwise_distance(animals[0], animals[x]) for x in range(4)]
# s2 = [F.pairwise_distance(animals[1], animals[x]) for x in range(4)]
# s3 = [F.pairwise_distance(animals[2], animals[x]) for x in range(4)]
# s4 = [F.pairwise_distance(animals[3], animals[x]) for x in range(4)]

In [ ]:
S = torch.Tensor([[torch.cosine_similarity(animals[i], animals[x]) for x in range(4)] for i in range(4)])

In [ ]:
# S = torch.Tensor([[F.pairwise_distance(animals[i], animals[x]) for x in range(4) if x != i] for i in range(4)])

In [ ]:
S

In [ ]:
# 让同一张图的不同view在latent space里靠近，不同图的view在latent space里远离，通过NT-Xent实现
Loss12 = S[0][1] / sum(S[0])
Loss21 = S[1][0] / sum(S[1])
Loss34 = S[2][3] / sum(S[2])
Loss43 = S[3][2] / sum(S[3])

In [ ]:
Loss12 + Loss21

In [ ]:
Loss34 + Loss43

In [ ]:
def L_distance(k):
    # k: [1,.., N]
    return S[2*k - 1-1][2*k-1] / torch.sum(S[2 *k-1-1]) + S[2*k - 1][2*k - 1-1] / torch.sum(S[2*k -1])

In [ ]:
L_distance(1)

In [ ]:
L_distance(2)

In [ ]:
loss = 0
for k in range(1, 3):
    loss += L_distance(k)

In [ ]:
loss / 4

**Apply NT-Xent in multi domains**

In [2]:
def load_image(file_path):
    image = Image.open(file_path).convert('RGB')
#     image = np.asarray(image)
    return image # PIL

In [3]:
data_dir = '/data/majie/Datasets/Domain-adaption/PACS/data'
doms = ['cartoon', 'photo', 'sketch']
doms_test = 'art_painting'
doms_test = 0
dom_datasets = {}
data_sizes = {}
image_datasets = {}
image_datasets_test = {}

In [4]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda', index=3)

In [6]:
for i, dom in enumerate(doms):
    image_datasets[dom] = {x: datasets.ImageFolder(os.path.join(data_dir, dom, x)) for x in ['train', 'val', 'test']}

In [7]:
test_dom_dataset = datasets.ImageFolder(osp.join(data_dir, 'art_painting', 'test'), transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

In [9]:
test_dom_dataset

Dataset ImageFolder
    Number of datapoints: 2048
    Root location: /data/majie/Datasets/Domain-adaption/PACS/data/art_painting/test
    StandardTransform
Transform: Compose(
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [14]:
test_dom_dataloader = DataLoader(test_dom_dataset, batch_size=16, shuffle=False)

In [10]:
test_data = test_dom_dataloader.__iter__().__next__()

In [49]:
inputs = test_data[0].to(device)

In [50]:
out = model_ft(inputs, 0.99)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 64 3 7 7, but got 3-dimensional input of size [3, 224, 224] instead

In [11]:
test_data[1]

tensor([0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
model_ft = resnet18()
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('./models/best_model_adversarial_contrastive.tar')['state'])
model_ft.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [1]:
# acc = 0
# for images, labels in test_dom_dataloader:
#     images = images.to(device)
#     labels = labels.to(device)
    
#     outputs = model_ft(images, 0.8)[1]
#     _, preds_classes = torch.max(outputs, 1)
#     print('preds_classes', preds_classes)
#     acc += torch.sum(preds_classes == labels.data)
# print('test on art domain, accuracy is: ', acc)

In [8]:
train, val = [], []
for idx, dom in enumerate(doms):
    train_label = [image_datasets[dom]['train'].imgs[i] + (idx,) for i in range(len(image_datasets[dom]['train'].imgs))]
    train.append(train_label)
    val_label = [image_datasets[dom]['val'].imgs[i] + (idx,) for i in range(len(image_datasets[dom]['val'].imgs))]
    val.append(val_label)
train = np.vstack(np.array(train))
val = np.vstack(np.array(val))
train[1839]

array(['/data/majie/Datasets/Domain-adaption/PACS/data/cartoon/train/6/pic_111.jpg',
       '6', '0'], dtype='<U84')

In [9]:
# image_datasets

In [10]:
class GenericDataset(Dataset):
    def __init__(self, dataset_root='', stage='train', random_sized_crop=False, num_imgs_per_cat=None):
        super(GenericDataset, self).__init__()
        
        # if split is not None, return the spcific split data
        # else return all data in source domains
        
        self.image_file_names = []
        self.stage = stage
        
        self.dataset = {'train': [], 'val': []}
        self.data = []
        
        if self.stage == 'test': # test domain
            
            dom = doms[doms_test] 
            # test split is the whole data in its domain
            self.data.extend(image_datasets[dom]['test'].imgs)
            
        else: # training domain
           
            self.dataset['train'] = train
            self.dataset['val'] = val
            
            self.data = self.dataset[self.stage]
       
            
          
    
    def __getitem__(self, index):
        # 1. return img - label
        image, class_label, dom_label = self.data[index]
        
        return image, class_label, dom_label

    
    def __len__(self):
        
        return len(self.data)

In [23]:
dataset = GenericDataset()

In [24]:
dataloader = DataLoader(dataset,batch_size=4)

In [25]:
data = dataloader.__iter__().__next__()

In [14]:
len(data)

3

In [11]:
data_transform = {
    'train': {
        'crop': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'color': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(1, 0.1, 0.05, 0.0), 
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    },
    'val': {
        'crop': transforms.Compose([
            transforms.CenterCrop((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'color': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ColorJitter(1, 0.1, 0.05, 0.0), 
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    },
    'test': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [12]:
class DataLoader_custom(object):
    def __init__(self,
                 dataset,
                 stage='train',
                 batch_size=8,
                 unsupervised=True,
                 epoch_size=None,
                 num_workers=8,
                 shuffle=True):
        
        self.dataset = dataset
        self.shuffle = shuffle
        self.epoch_size = epoch_size if epoch_size is not None else len(dataset)
        self.batch_size = batch_size
        self.unsupervised = unsupervised
        self.num_workers = num_workers
        
        self.stage = stage

        mean_pix  = [0.485, 0.456, 0.406]
        std_pix = [0.229, 0.224, 0.225]
        
        
        self.transform = data_transform
     

    def get_iterator(self, epoch=0):
        rand_seed = epoch * self.epoch_size
        random.seed(rand_seed)
        # if in unsupervised mode define a loader function that given the
        # index of an image it returns the 4 rotated copies of the image
        # plus the label of the rotation, i.e., 0 for 0 degrees rotation,
        # 1 for 90 degrees, 2 for 180 degrees, and 3 for 270 degrees.
        
        if self.stage == 'test':
            def _load_function(idx):
#                 idx = idx % len(self.dataset)
                img0, class_label, dom_label = self.dataset[idx]
                img0 = load_image(img0)
                image = self.transform[self.stage](img0)
               
                class_label = torch.LongTensor(int(class_label))
                dom_label = torch.LongTensor(int(dom_label))
                
#                 print('--------------------------')

                return image, (class_label, dom_label)

            def _collate_fun(batch):
                return default_collate(batch)
        
        else:
            def _load_function(idx):
                idx = idx % len(self.dataset)
                img0, class_label, dom_label = self.dataset[idx]

                img0 = load_image(img0)
                rotated_imgs = [
                    self.transform[self.stage]['crop'](img0),
                    self.transform[self.stage]['color'](img0),
#                     transform(rotate_img(img0,  90)),
#                     transform(rotate_img(img0, 180)),
#                     transform(rotate_img(img0, 270))
                ]
                labels = [(int(class_label), int(dom_label)) for _ in range(2)]
                rotation_labels = torch.LongTensor(labels)
                return torch.stack(rotated_imgs, dim=0), rotation_labels

            def _collate_fun(batch):
                batch = default_collate(batch)
                assert(len(batch)==2)
#                 print('batch[1]', batch[1])
                batch_size, rotations, channels, height, width = batch[0].size()
                batch[0] = batch[0].view([batch_size*rotations, channels, height, width])
                batch[1] = batch[1].view((batch_size*rotations, 2))
                return batch


        tnt_dataset = tnt.dataset.ListDataset(elem_list=range(self.epoch_size),
            load=_load_function)
        data_loader = tnt_dataset.parallel(batch_size=self.batch_size,
            collate_fn=_collate_fun, num_workers=self.num_workers,
            shuffle=self.shuffle)
        return data_loader

    def __call__(self, epoch=0):
        return self.get_iterator(epoch)

    def __len__(self):
        return self.epoch_size / self.batch_size

In [13]:
dataloader = {x: DataLoader_custom(GenericDataset(stage=x), stage=x) for x in ['train', 'val', 'test']}
data_size = {x: len(GenericDataset(stage=x)) for x in ['train', 'val', 'test']}
data_size

{'train': 7137, 'val': 806, 'test': 2344}

In [14]:
len(dataloader['train']())

893

In [15]:
class DataPrefetcher():
    def __init__(self, loader):
        self.loader = iter(loader)
        self.stream = torch.cuda.Stream()
        # With Amp, it isn't necessary to manually convert data to half.
        # if args.fp16:
        #     self.mean = self.mean.half()
        #     self.std = self.std.half()
        self.preload()

    def preload(self):
        try:
            self.batch = next(self.loader)
        except StopIteration:
            self.batch = None
            return
        with torch.cuda.stream(self.stream):
            for k in range(len(self.batch)):
                if k != 'meta':
                    self.batch[k] = self.batch[k].to(device=device, non_blocking=True)

            # With Amp, it isn't necessary to manually convert data to half.
            # if args.fp16:
            #     self.next_input = self.next_input.half()
            # else:
            #     self.next_input = self.next_input.float()

    def next(self):
        torch.cuda.current_stream().wait_stream(self.stream)
        batch = self.batch
        self.preload()
        return batch

In [16]:
prefetcher = DataPrefetcher(dataloader['train']())
image, label = prefetcher.next()

image.size()


torch.Size([16, 3, 224, 224])

**TODO:** check the dataloader order as above according to Paper: A Simple Framework for Contrastive Learning of Visual Representations

**REFER:** [adverserial training](https://zhuanlan.zhihu.com/p/50710267)

In [17]:
def contrastive_distance(k):
    # k: [1,.., N]
    return -torch.log(torch.exp(S[2*k - 1-1][2*k-1]) / torch.sum(torch.exp(S[2 *k-1-1]))\
+ torch.exp(S[2*k - 1][2*k - 1-1]) / torch.sum(torch.exp(S[2*k -1])))

**TODO  alpha setting:** [refer](https://github.com/ShichengChen/Domain-Adversarial-Training-of-Neural-Networks/blob/master/mnist.ipynb)

In [18]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    best_model_wts = copy.deepcopy(model.state_dict())
    dataloader = {x: DataLoader_custom(GenericDataset(stage=x), stage=x) for x in ['train', 'val', 'test']}
    data_size = {x: len(GenericDataset(stage=x)) for x in ['train', 'val', 'test']}
    best_acc = 0.0
    alpha = 0.0


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 20)
        
        i = 0
     
        for phase in ['train', 'val']:
            len_dataloader = len(dataloader[phase](epoch))
            dom_data_size = data_size[phase]
            if phase == 'train':
                total_steps = num_epochs * len_dataloader
                start_steps = epoch * len_dataloader
                p = float(i + start_steps) / total_steps
                alpha = 2. / (1. + np.exp(-10 * p)) - 1
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            ## train each dom, sum over all losses, and backward the total loss

            doms_data_size = 0
            # prefetcher = DataPrefetcher(dataloader['train']())
            dom_data_loader = DataPrefetcher(dataloader[phase](epoch))
            
            
            
            batch = dom_data_loader.next()
            while batch is not None:
                inputs, labels  = batch

#             for inputs, labels in dom_data_loader.next():
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

                label_class = labels[:, 0]
                label_dom = labels[:, 1]

                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs, alpha) # 0: contrastive_feature, 1: class_label, 2: dom_label

                    batch_data_size = inputs.size(0)
                    
                    k_len = batch_data_size / 2 + 1
                    
                    contrastive_features = outputs[0].view(batch_data_size, 1, 128)
                    S = torch.Tensor([[torch.cosine_similarity(contrastive_features[i], contrastive_features[x])\
                                       for x in range(batch_data_size)] for i in range(batch_data_size)])

                    contrastive_loss = 0
                    for k in range(1, int(k_len)):
                        contrastive_distance =  -torch.log(torch.exp(S[2*k - 1-1][2*k-1]) / torch.sum(torch.exp(S[2 *k-1-1]))\
+ torch.exp(S[2*k - 1][2*k - 1-1]) / torch.sum(torch.exp(S[2*k -1])))
                        contrastive_loss += contrastive_distance

                    contrastive_loss = contrastive_loss.to(device)
                    loss_class = criterion(outputs[1], label_class)
                    loss_dom = criterion(outputs[2], label_dom)

                    total_loss = 0.6 * (contrastive_loss + loss_class) + 0.4 * loss_dom

                    _, preds_classes = torch.max(outputs[1], 1)
                    _, preds_dom = torch.max(outputs[2], 1)


                    if phase == 'train':
                        i += 1
                        total_loss.backward()
                        optimizer.step()
                    batch = dom_data_loader.next()

                running_loss += total_loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds_classes == label_class.data)
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dom_data_size / 2
            epoch_acc = running_corrects.double() / dom_data_size / 2

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            


    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    if not os.path.exists('./models'):
        os.makedirs('./models')

    outfile = os.path.join('./models', 'best_model_adversarial_contrastive_0.4.tar')
    torch.save({'state': best_model_wts, 'alpha': alpha}, outfile)
    return model

In [19]:
model = resnet18()

In [20]:
# model = nn.DataParallel(model, device_ids=[0,1,2,3])

model = model.to(device)
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [21]:
device

device(type='cuda', index=3)

In [22]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=20)

Epoch 0/20
--------------------
train Loss: 10.6327 Acc: 0.5042
val Loss: 9.8574 Acc: 0.7128
Epoch 1/20
--------------------
train Loss: 10.3828 Acc: 0.6580
val Loss: 9.7494 Acc: 0.7854
Epoch 2/20
--------------------
train Loss: 10.2896 Acc: 0.7136
val Loss: 9.8828 Acc: 0.7723
Epoch 3/20
--------------------
train Loss: 10.2342 Acc: 0.7456
val Loss: 9.8158 Acc: 0.7860
Epoch 4/20
--------------------
train Loss: 10.2271 Acc: 0.7713
val Loss: 9.7671 Acc: 0.8313
Epoch 5/20
--------------------
train Loss: 10.2099 Acc: 0.7927
val Loss: 9.8347 Acc: 0.8220
Epoch 6/20
--------------------
train Loss: 10.2038 Acc: 0.8064
val Loss: 9.7708 Acc: 0.8412
Epoch 7/20
--------------------
train Loss: 10.1053 Acc: 0.8649
val Loss: 9.6667 Acc: 0.8989
Epoch 8/20
--------------------
train Loss: 10.0582 Acc: 0.8854
val Loss: 9.6605 Acc: 0.9119
Epoch 9/20
--------------------
train Loss: 10.0370 Acc: 0.8936
val Loss: 9.6501 Acc: 0.9076
Epoch 10/20
--------------------
train Loss: 10.0195 Acc: 0.9002
val L

In [24]:
test_dom_dataset = datasets.ImageFolder(osp.join(data_dir, 'art_painting', 'test'), transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

In [25]:
test_dom_dataloader = DataLoader(test_dom_dataset, batch_size=16, shuffle=False)

In [26]:
acc = 0
for images, labels in test_dom_dataloader:
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images, 0.8)[1]
    _, preds_classes = torch.max(outputs, 1)
    
    acc += torch.sum(preds_classes == labels.data)
    print('Accuarcy ', acc)
print('test on art domain, accuracy is: ', acc)

Accuarcy  tensor(10, device='cuda:3')
Accuarcy  tensor(21, device='cuda:3')
Accuarcy  tensor(34, device='cuda:3')
Accuarcy  tensor(44, device='cuda:3')
Accuarcy  tensor(55, device='cuda:3')
Accuarcy  tensor(68, device='cuda:3')
Accuarcy  tensor(78, device='cuda:3')
Accuarcy  tensor(87, device='cuda:3')
Accuarcy  tensor(95, device='cuda:3')
Accuarcy  tensor(105, device='cuda:3')
Accuarcy  tensor(115, device='cuda:3')
Accuarcy  tensor(122, device='cuda:3')
Accuarcy  tensor(129, device='cuda:3')
Accuarcy  tensor(136, device='cuda:3')
Accuarcy  tensor(147, device='cuda:3')
Accuarcy  tensor(156, device='cuda:3')
Accuarcy  tensor(165, device='cuda:3')
Accuarcy  tensor(173, device='cuda:3')
Accuarcy  tensor(181, device='cuda:3')
Accuarcy  tensor(189, device='cuda:3')
Accuarcy  tensor(195, device='cuda:3')
Accuarcy  tensor(202, device='cuda:3')
Accuarcy  tensor(212, device='cuda:3')
Accuarcy  tensor(220, device='cuda:3')
Accuarcy  tensor(233, device='cuda:3')
Accuarcy  tensor(242, device='cuda

In [ ]:
# multi GPU train, to load saved weights
def load_network(network):
    save_path = os.path.join('./model',name,'net_%s.pth'%opt.which_epoch)
    state_dict = torch.load(save_path)
    # create new OrderedDict that does not contain `module.`
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        namekey = k[7:] # remove `module.`
        new_state_dict[namekey] = v
    # load params
    network.load_state_dict(new_state_dict)
    return network



**TODO:**
mutilple GPU train